In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path
from sklearn.model_selection import train_test_split


Create 2 layered NN

In [2]:
def create_model(inputs, weights, biases):
    '''Build the model'''
    
    #Create 2-layered NN
    input_layer= tf.matmul(inputs,weights['input'])
    # Hidden layer 1 (relu activation)
    hidden_layer1= tf.nn.relu(input_layer + biases['input'])
    # Hidden layer 2 (relu activation)
    hidden_layer2= tf.nn.relu(tf.matmul(hidden_layer1,weights['hidden1'])+biases['hidden1'])
    # Output layer (linear activation)
    output_layer= tf.matmul(hidden_layer2, weights['hidden2']) + biases['hidden2']

    return output_layer


In [3]:
#First Column is the label and others are the features
COLUMNS = ['className','handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution',
'physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban',
'aid-to-nicaraguan-contras','mx-missle','immigration','synfuels-corporation-cutback','education-spending',
'superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']

# Read data into CSV file and convert ? to NaN
df = pd.read_csv(os.path.join("./house-votes-84.data.txt")
                       , names=COLUMNS
                       , skipinitialspace=True
                       , na_values="?")

In [4]:
df

,className,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,NaN,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,NaN,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,NaN,NaN


Dealing with missing data (NaN)
There are better ways to deal with misssing data. Here, we chose the easy way by removing the rows that contain NaN values 

In [5]:
#Rows with NaN values are removed from data
df.replace(["NaN"], np.nan, inplace = True)
df = df.dropna()
df = df.reset_index(drop=True)
  
#New row numbers after removing NaN values
rows_number=df.shape[0]

In [6]:
df

,className,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
3,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,y
4,democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y
5,democrat,y,n,y,n,n,n,y,y,y,n,y,n,n,n,y,y
6,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
7,republican,y,n,n,y,y,n,y,y,y,n,n,y,y,y,n,y
8,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
9,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,n


In [7]:
#Encode the label column republican/democrat to 1 and 0
for i in range(0,len(COLUMNS)):
    cleanup_nums = {COLUMNS[i]: {"y":1, "n":0, "republican":1,"democrat":0}}
    df.replace(cleanup_nums, inplace=True)

One hot encoding of labels

In [8]:
#Make a 2 column matrix for the labels column
labels=np.zeros((rows_number, 2))
label=df['className']
for i in range(0,len(label)):
    if(label[i]==0):
        labels[i,0]=0
        labels[i,1]=1
    else:
        labels[i,0]=1
        labels[i,1]=0

In [9]:
labels

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.

In [10]:
inputs=df[COLUMNS[2:]]
inputs=inputs.values

In [11]:
x_train, x_test, y_train, y_test = train_test_split(inputs, labels)


In [12]:
#HyperParameters Training
learning_rate = 0.5
epochs = 50
batch_size = 50 #small dataset

n_features = 15
n_calsses=2

#HyperParameters NN
hidden_nodes_layer1=10
hidden_nodes_layer2=10


In [13]:
#Define variables for weights and biases
input_weights = tf.Variable(tf.random_normal([n_features, hidden_nodes_layer1]))
input_biases = tf.Variable(tf.zeros([1,hidden_nodes_layer1]))

hidden1_weights = tf.Variable(tf.random_normal([hidden_nodes_layer1, hidden_nodes_layer2]))
hidden1_biases = tf.Variable(tf.zeros([1,hidden_nodes_layer2]))

hidden2_weights = tf.Variable(tf.random_normal([hidden_nodes_layer2, n_calsses]))
hidden2_biases = tf.Variable(tf.zeros([1,n_calsses]))

weights={'input':input_weights,'hidden1':hidden1_weights,'hidden2':hidden2_weights}
biases={'input':input_biases,'hidden1':hidden1_biases,'hidden2':hidden2_biases}

In [14]:
#Placeholders to initialize 
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_calsses])

#Create the NN model
prediction=create_model(x,weights,biases)


In [15]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [16]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#Execute the default graph

with tf.Session() as sess:
    sess.run(init)    
    #Training cycle
    # Loop epochs
    for epoch in range(epochs):
        avg_cost = 0       
        total_batch = int(len(x_train)/batch_size)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization and loss 
            _, cost = sess.run([optimizer, loss_function], feed_dict={x: batch_x,
                                                          y: batch_y})
            # average cost
            avg_cost += cost / total_batch
        print("Epoch:", epoch+1, "cost=", avg_cost)
    
    #accuracy
    print("Accuracy: " + str(accuracy.eval(feed_dict={x: x_test,
                                                          y: y_test})))



('Epoch:', 1, 'cost=', 8.35253159205119)
('Epoch:', 2, 'cost=', 4.48852202296257)
('Epoch:', 3, 'cost=', 0.41344955563545227)
('Epoch:', 4, 'cost=', 0.3711117208003998)
('Epoch:', 5, 'cost=', 0.34320303797721863)
('Epoch:', 6, 'cost=', 0.3173873523871104)
('Epoch:', 7, 'cost=', 0.29431187113126117)
('Epoch:', 8, 'cost=', 0.2699783593416214)
('Epoch:', 9, 'cost=', 0.25346950193246204)
('Epoch:', 10, 'cost=', 0.23797039687633514)
('Epoch:', 11, 'cost=', 0.22957853972911835)
('Epoch:', 12, 'cost=', 0.21710728605588275)
('Epoch:', 13, 'cost=', 0.20728754997253415)
('Epoch:', 14, 'cost=', 0.19315277536710104)
('Epoch:', 15, 'cost=', 0.18193120261033374)
('Epoch:', 16, 'cost=', 0.17563529312610626)
('Epoch:', 17, 'cost=', 0.1700537552436193)
('Epoch:', 18, 'cost=', 0.16498331725597382)
('Epoch:', 19, 'cost=', 0.15749090909957886)
('Epoch:', 20, 'cost=', 0.15694440404574075)
('Epoch:', 21, 'cost=', 0.14931955933570862)
('Epoch:', 22, 'cost=', 0.1407139052947362)
('Epoch:', 23, 'cost=', 0.1342